Implement CNNs using Adam and RMSprop optimizers with a learning rate of 0.001 on Peach 
images. Record validation loss and accuracy.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import os

# Set parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001

# Update these paths according to your dataset structure
# Assumes structure: peach_dataset/train/[classes], peach_dataset/val/[classes]
train_dir = '/home/adithya/Deep_Learning/DL3_Dataset/Peach/Train'
val_dir = '/home/adithya/Deep_Learning/DL3_Dataset/Peach/Val'

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)
val_datagen = ImageDataGenerator(rescale=1./255)

def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def plot_history(histories, optimizer_names):
    plt.figure(figsize=(12, 5))
    for i, history in enumerate(histories):
        plt.subplot(1, 2, 1)
        plt.plot(history.history['val_accuracy'], label=optimizer_names[i])
        plt.title('Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(history.history['val_loss'], label=optimizer_names[i])
        plt.title('Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    if not os.path.exists(train_dir):
        raise FileNotFoundError(f"Training directory '{train_dir}' does not exist. Please check your dataset path.")
    class_folders = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]
    if not class_folders:
        raise ValueError(f"No class subfolders found in '{train_dir}'. Please organize your dataset as '{train_dir}/class_name/'.")
    num_classes = len(class_folders)
    print(f'Number of classes: {num_classes}')

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    optimizers_list = [
        ('Adam', optimizers.Adam(learning_rate=LEARNING_RATE)),
        ('RMSprop', optimizers.RMSprop(learning_rate=LEARNING_RATE))
    ]
    histories = []
    for name, optimizer in optimizers_list:
        print(f"\nTraining with optimizer: {name}")
        model = build_cnn_model((IMG_HEIGHT, IMG_WIDTH, 3), num_classes)
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        history = model.fit(
            train_generator,
            epochs=EPOCHS,
            validation_data=val_generator,
            verbose=1
        )
        histories.append(history)

    plot_history(histories, [name for name, _ in optimizers_list])

    for i, (name, _) in enumerate(optimizers_list):
        val_acc = histories[i].history['val_accuracy'][-1]
        val_loss = histories[i].history['val_loss'][-1]
        print(f"{name}: Final Validation Accuracy = {val_acc:.4f}, Final Validation Loss = {val_loss:.4f}")